## Setup and Data Collection

In [2]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 


import psycopg2 as pg
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from cred.cred_user import username
from cred.cred_p import pgpass
from cred.cred_host import host_loc
from cred.cred_port import cred_port

db = "Proj_ETL"

In [3]:
neuro_data = "../data_large/Neuroscience_Data.csv"
neuro_df = pd.read_csv(neuro_data)
neuro_df.head()

,Unnamed: 0,citation_abstract,citation_author,citation_author_email,citation_author_institution,citation_date,citation_doi,citation_issn,citation_journal_abbrev,citation_journal_title,...,citation_publisher,citation_title,citation_volume,dc.identifier,description,img,keywords,text,title,url
0,0,The peripheral nervous system has an intrinsic...,"Offen, Daniel",danioffen@gmail.com,"Sackler Faculty of Medicine, Tel Aviv Universi...",2019.0,10.3389/fnins.2019.00151,1662-453X,Front. Neurosci.,Frontiers in Neuroscience,...,Frontiers,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,13.0,NaN,The peripheral nervous system has an intrinsic...,https://www.frontiersin.org/files/MyHome%20Art...,Human muscle progenitor cells; Neurotrophic fa...,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,https://www.frontiersin.org/articles/10.3389/f...
1,1,"In this study, we explore the functional role ...","Molkov, Yaroslav I.",NaN,"Department of Mathematics and Statistics, Geor...",2019.0,10.3389/fncir.2019.00010,1662-5110,Front. Neural Circuits,Frontiers in Neural Circuits,...,Frontiers,The Functional Role of Striatal Cholinergic In...,13.0,doi:10.3389/fncir.2019.00010,"In this study, we explore the functional role ...",https://www.frontiersin.org/files/MyHome%20Art...,Striatum; Basal Ganglia; Reinforcement Learn...,"In this study, we explore the functional role ...",The Functional Role of Striatal Cholinergic In...,https://www.frontiersin.org/articles/10.3389/f...
2,2,Recent studies clearly indicate that endocrine...,"Herman-Antosiewicz, Anna",anna.herman@biol.ug.edu.pl,"University of Gdansk, Gdansk, 80-952, Pomerani...",2019.0,10.3389/fnins.2019.00165,1662-453X,Front. Neurosci.,Frontiers in Neuroscience,...,Frontiers,Iron Metabolism of the Skeletal Muscle and Neu...,13.0,NaN,Recent studies clearly indicate that endocrine...,https://www.frontiersin.org/files/MyHome%20Art...,"myokine, ; Iron,; ALS, amyotrophic lateral scl...",Iron Metabolism of the Skeletal Muscle and Neu...,Iron Metabolism of the Skeletal Muscle and Neu...,https://www.frontiersin.org/articles/10.3389/f...
3,3,Emerging neuroimaging studies emphasize the dy...,"Wang, Zheng",zheng.wang@ion.ac.cn,"Institute of Neuroscience, Shanghai Institutes...",2019.0,10.3389/fnins.2019.00174,1662-453X,Front. Neurosci.,Frontiers in Neuroscience,...,Frontiers,Brain Map of Intrinsic Functional Flexibility ...,13.0,NaN,Emerging neuroimaging studies emphasize the dy...,https://www.frontiersin.org/files/MyHome%20Art...,dynamic brain organization; intrinsic function...,Brain Map of Intrinsic Functional Flexibility ...,Brain Map of Intrinsic Functional Flexibility ...,https://www.frontiersin.org/articles/10.3389/f...
4,4,The disruption of iron metabolism and iron tra...,"So, Po-Wah",Po-wah.so@kcl.ac.uk,"King's College London, London, WC2R 2LS, Unite...",2019.0,10.3389/fnins.2019.00181,1662-453X,Front. Neurosci.,Frontiers in Neuroscience,...,Frontiers,Low CSF melanotransferrin is associated with c...,13.0,NaN,The disruption of iron metabolism and iron tra...,https://www.frontiersin.org/files/MyHome%20Art...,Alzheimer’s disease; CSF; Iron; MCI; Melanotra...,Low CSF melanotransferrin is associated with c...,Low CSF melanotransferrin is associated with c...,https://www.frontiersin.org/articles/10.3389/f...


In [4]:
mental_data = "../data/Mental_Data.csv"
mental_df = pd.read_csv(mental_data)
mental_df.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision)
0,Afghanistan; 2016,NaN,Yes,1987,Yes,2016.0
1,Afghanistan; 2014,NaN,Yes,NaN,Yes,NaN
2,Albania; 2016,NaN,Yes,2012,Yes,2013.0
3,Albania; 2014,NaN,Yes,NaN,Yes,NaN
4,Algeria; 2016,NaN,Yes,1905,No,NaN


In [25]:
journal_data = "../data_large/Journal_Data.csv"
journal_df = pd.read_csv(journal_data, sep=';')
journal_df.head()

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Quartile,H index,Total Docs. (2018),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Coverage,Categories
0,1,17437,Nature Reviews Neuroscience,journal,"14710048, 1471003X","21,210",Q1,375,136,401,9159,6587,194,"32,24","67,35",United Kingdom,Nature Publishing Group,2000-ongoing,Neuroscience (miscellaneous) (Q1)
1,2,17436,Nature Neuroscience,journal,10976256,"15,852",Q1,375,248,815,11676,13577,712,"17,81","47,08",United Kingdom,Nature Publishing Group,1998-ongoing,Neuroscience (miscellaneous) (Q1)
2,3,14181,Annual Review of Neuroscience,book series,"0147006X, 15454126","12,626",Q1,224,26,67,3689,1113,67,"12,02","141,88",United States,Annual Reviews Inc.,1978-ongoing,Neuroscience (miscellaneous) (Q1)
3,4,17978,Neuron,journal,08966273,"11,377",Q1,430,515,1560,31105,18464,1488,"11,18","60,40",United States,Cell Press,1988-ongoing,Neuroscience (miscellaneous) (Q1)
4,5,17435,EMBO Journal,journal,"02614189, 14602075","7,579",Q1,368,259,754,14304,5989,660,"9,96","55,23",Germany,Wiley-Blackwell,1982-ongoing,"Biochemistry, Genetics and Molecular Biology (..."


## Exploring the Neuroscience Data

In [4]:
neuro_df.head(1)

,Unnamed: 0,citation_abstract,citation_author,citation_author_email,citation_author_institution,citation_date,citation_doi,citation_issn,citation_journal_abbrev,citation_journal_title,...,citation_publisher,citation_title,citation_volume,dc.identifier,description,img,keywords,text,title,url
0,0,The peripheral nervous system has an intrinsic...,"Offen, Daniel",danioffen@gmail.com,"Sackler Faculty of Medicine, Tel Aviv Universi...",2019.0,10.3389/fnins.2019.00151,1662-453X,Front. Neurosci.,Frontiers in Neuroscience,...,Frontiers,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,13.0,NaN,The peripheral nervous system has an intrinsic...,https://www.frontiersin.org/files/MyHome%20Art...,Human muscle progenitor cells; Neurotrophic fa...,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,https://www.frontiersin.org/articles/10.3389/f...


In [5]:
neuro_df.columns

Index(['Unnamed: 0', 'citation_abstract', 'citation_author',
       'citation_author_email', 'citation_author_institution', 'citation_date',
       'citation_doi', 'citation_issn', 'citation_journal_abbrev',
       'citation_journal_title', 'citation_keywords', 'citation_language',
       'citation_online_date', 'citation_pages', 'citation_pdf_url',
       'citation_publication_date', 'citation_publisher', 'citation_title',
       'citation_volume', 'dc.identifier', 'description', 'img', 'keywords',
       'text', 'title', 'url'],
      dtype='object')

In [6]:
neuro_df["citation_journal_title"].value_counts()

Frontiers in Human Neuroscience            5330
Frontiers in Neuroscience                  4252
Frontiers in Cellular Neuroscience         2719
Frontiers in Computational Neuroscience    1120
Frontiers in Neural Circuits                966
Frontiers in Neuroanatomy                   953
Frontiers in Neuroinformatics               500
Frontiers in Neurorobotics                  277
Frontiers in Neuroengineering               117
Frontiers in Neuroenergetics                 63
Frontiers in Artificial Intelligence          1
Name: citation_journal_title, dtype: int64

In [7]:
neuro_df["citation_date"].value_counts()

2018.0    2530
2014.0    2512
2017.0    2349
2015.0    2241
2016.0    2106
2013.0    2052
2012.0     955
2019.0     600
2011.0     562
2010.0     391
Name: citation_date, dtype: int64

In [8]:
neuro_df["citation_author_institution"].value_counts().head()

Center for Collaboration and Innovation in Brain and Learning Sciences, Beijing Normal University, Beijing, China    14
Institute of Neuroinformatics, University of Zurich and ETH Zurich, Zurich, Switzerland                              10
University College London, London, United Kingdom                                                                    10
Center for Behavioral Brain Sciences, Magdeburg, Germany                                                              8
Institute for Human Movement and Medical Sciences, Niigata University of Health and Welfare, Niigata, Japan           8
Name: citation_author_institution, dtype: int64

## Exploring the Mental Health Policy Data

In [9]:
mental_df.head(1)

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision)
0,Afghanistan; 2016,NaN,Yes,1987,Yes,2016.0


In [6]:
mental_df["Government expenditures on mental hospitals" +
          " as a percentage of total government expenditures on mental health (%)"].describe()

count    78.000000
mean      3.717949
std       3.608854
min       0.000000
25%       1.100000
50%       2.700000
75%       5.525000
max      19.500000
Name: Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%), dtype: float64

In [10]:
mental_df["Stand-alone law for mental health"].value_counts()

Yes      208
No       126
Yes        2
No         1
2018       1
No No      1
yes        1
Name: Stand-alone law for mental health, dtype: int64

In [7]:
mental_df["Year the law was enacted (latest revision)"].value_counts().head(10)

2016    11
2017     9
2013     8
2001     6
2011     5
2015     5
2010     4
2009     4
2008     4
2014     4
Name: Year the law was enacted (latest revision), dtype: int64

In [8]:
mental_df["Publication year of the policy or plan (latest revision)"].value_counts().head(10)

2016.0    30
2013.0    16
2015.0    16
2014.0    15
2012.0    13
2017.0    13
2011.0    10
2008.0     3
2004.0     3
2009.0     3
Name: Publication year of the policy or plan (latest revision), dtype: int64

In [11]:
mental_df["Stand-alone policy or plan for mental health"].value_counts()

Yes        268
No          73
Yes Yes      1
YES          1
Name: Stand-alone policy or plan for mental health, dtype: int64

In [9]:
mental_df["Publication year of the policy or plan (latest revision)"].value_counts().head(10)

2016.0    30
2013.0    16
2015.0    16
2014.0    15
2012.0    13
2017.0    13
2011.0    10
2008.0     3
2004.0     3
2009.0     3
Name: Publication year of the policy or plan (latest revision), dtype: int64

## Exploring the Journal Data

In [11]:
journal_df.head(1)

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Quartile,H index,Total Docs. (2018),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Coverage,Categories
0,1,17437,Nature Reviews Neuroscience,journal,"14710048, 1471003X","21,210",Q1,375,136,401,9159,6587,194,"32,24","67,35",United Kingdom,Nature Publishing Group,2000-ongoing,Neuroscience (miscellaneous) (Q1)


In [12]:
journal_df.columns

Index(['Rank', 'Sourceid', 'Title', 'Type', 'Issn', 'SJR', 'SJR Quartile',
       'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
       'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'Country', 'Publisher',
       'Coverage', 'Categories'],
      dtype='object')

In [13]:
relevant_journals = list(neuro_df["citation_journal_title"].unique())
relevant_journals

['Frontiers in Neuroscience',
 'Frontiers in Neural Circuits',
 'Frontiers in Human Neuroscience',
 'Frontiers in Neuroanatomy',
 'Frontiers in Cellular Neuroscience',
 'Frontiers in Neuroinformatics',
 'Frontiers in Neurorobotics',
 'Frontiers in Computational Neuroscience',
 'Frontiers in Neuroengineering',
 'Frontiers in Neuroenergetics',
 'Frontiers in Artificial Intelligence',
 nan]

In [18]:
for journal in journal_df['Title']:
    if journal in relevant_journals:
        print(journal)

Frontiers in Neural Circuits
Frontiers in Neuroinformatics
Frontiers in Neuroanatomy
Frontiers in Neuroscience
Frontiers in Computational Neuroscience
Frontiers in Neuroengineering


In [15]:
journal_df["Total Docs. (3years)"].describe()

count     149.000000
mean      449.912752
std       687.963288
min         1.000000
25%        88.000000
50%       238.000000
75%       469.000000
max      3751.000000
Name: Total Docs. (3years), dtype: float64

In [16]:
journal_df["Total Refs."].describe()

count       149.000000
mean       8836.510067
std       15328.293367
min           0.000000
25%         885.000000
50%        3689.000000
75%        9848.000000
max      110729.000000
Name: Total Refs., dtype: float64

In [17]:
journal_df["Country"].value_counts().head(20)

United States           45
United Kingdom          30
Netherlands             23
Germany                 11
Switzerland              9
India                    5
Brazil                   3
United Arab Emirates     3
Egypt                    2
Russian Federation       2
New Zealand              2
Greece                   2
China                    1
Slovakia                 1
Turkey                   1
Poland                   1
Portugal                 1
Italy                    1
Japan                    1
Spain                    1
Name: Country, dtype: int64

## Choosing the Columns of interest

### neuro

In [14]:
neuro_df.columns

Index(['Unnamed: 0', 'citation_abstract', 'citation_author',
       'citation_author_email', 'citation_author_institution', 'citation_date',
       'citation_doi', 'citation_issn', 'citation_journal_abbrev',
       'citation_journal_title', 'citation_keywords', 'citation_language',
       'citation_online_date', 'citation_pages', 'citation_pdf_url',
       'citation_publication_date', 'citation_publisher', 'citation_title',
       'citation_volume', 'dc.identifier', 'description', 'img', 'keywords',
       'text', 'title', 'url'],
      dtype='object')

In [19]:
neuro_df = neuro_df[["title","citation_author","citation_date",\
                             "citation_journal_title","description","keywords",]]
neuro_df.head()

,title,citation_author,citation_date,citation_journal_title,description,keywords
0,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,"Offen, Daniel",2019.0,Frontiers in Neuroscience,The peripheral nervous system has an intrinsic...,Human muscle progenitor cells; Neurotrophic fa...
1,The Functional Role of Striatal Cholinergic In...,"Molkov, Yaroslav I.",2019.0,Frontiers in Neural Circuits,"In this study, we explore the functional role ...",Striatum; Basal Ganglia; Reinforcement Learn...
2,Iron Metabolism of the Skeletal Muscle and Neu...,"Herman-Antosiewicz, Anna",2019.0,Frontiers in Neuroscience,Recent studies clearly indicate that endocrine...,"myokine, ; Iron,; ALS, amyotrophic lateral scl..."
3,Brain Map of Intrinsic Functional Flexibility ...,"Wang, Zheng",2019.0,Frontiers in Neuroscience,Emerging neuroimaging studies emphasize the dy...,dynamic brain organization; intrinsic function...
4,Low CSF melanotransferrin is associated with c...,"So, Po-Wah",2019.0,Frontiers in Neuroscience,The disruption of iron metabolism and iron tra...,Alzheimer’s disease; CSF; Iron; MCI; Melanotra...


### mental

In [18]:
mental_df.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision)
0,Afghanistan; 2016,NaN,Yes,1987,Yes,2016.0
1,Afghanistan; 2014,NaN,Yes,NaN,Yes,NaN
2,Albania; 2016,NaN,Yes,2012,Yes,2013.0
3,Albania; 2014,NaN,Yes,NaN,Yes,NaN
4,Algeria; 2016,NaN,Yes,1905,No,NaN


### journal

In [20]:
journal_df.head()

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Quartile,H index,Total Docs. (2018),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Publisher,Coverage,Categories
0,1,17437,Nature Reviews Neuroscience,journal,"14710048, 1471003X","21,210",Q1,375,136,401,9159,6587,194,"32,24","67,35",United Kingdom,Nature Publishing Group,2000-ongoing,Neuroscience (miscellaneous) (Q1)
1,2,17436,Nature Neuroscience,journal,10976256,"15,852",Q1,375,248,815,11676,13577,712,"17,81","47,08",United Kingdom,Nature Publishing Group,1998-ongoing,Neuroscience (miscellaneous) (Q1)
2,3,14181,Annual Review of Neuroscience,book series,"0147006X, 15454126","12,626",Q1,224,26,67,3689,1113,67,"12,02","141,88",United States,Annual Reviews Inc.,1978-ongoing,Neuroscience (miscellaneous) (Q1)
3,4,17978,Neuron,journal,08966273,"11,377",Q1,430,515,1560,31105,18464,1488,"11,18","60,40",United States,Cell Press,1988-ongoing,Neuroscience (miscellaneous) (Q1)
4,5,17435,EMBO Journal,journal,"02614189, 14602075","7,579",Q1,368,259,754,14304,5989,660,"9,96","55,23",Germany,Wiley-Blackwell,1982-ongoing,"Biochemistry, Genetics and Molecular Biology (..."


In [21]:
journal_df.columns

Index(['Rank', 'Sourceid', 'Title', 'Type', 'Issn', 'SJR', 'SJR Quartile',
       'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
       'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'Country', 'Publisher',
       'Coverage', 'Categories'],
      dtype='object')

In [26]:
journal_df = journal_df[["Title","Rank","Total Docs. (3years)",\
                             "Total Refs.","Citable Docs. (3years)","Country"]]
journal_df.rename(columns = {"Title":"title"}, inplace = True)
journal_df.head()

,title,Rank,Total Docs. (3years),Total Refs.,Citable Docs. (3years),Country
0,Nature Reviews Neuroscience,1,401,9159,194,United Kingdom
1,Nature Neuroscience,2,815,11676,712,United Kingdom
2,Annual Review of Neuroscience,3,67,3689,67,United States
3,Neuron,4,1560,31105,1488,United States
4,EMBO Journal,5,754,14304,660,Germany


### merging idea

In [27]:
neuro_journal_merge = pd.merge(neuro_df, journal_df, how = "inner", on = "title")
neuro_journal_merge.groupby(["title"]).head(1).sort_values(["Rank"])

,title,citation_author,citation_date,citation_journal_title,description,keywords,Rank,Total Docs. (3years),Total Refs.,Citable Docs. (3years),Country
40,Frontiers in Neural Circuits,"Mitterauer, Bernhard J",2013.0,Frontiers in Neural Circuits,Abstract We examine the interactions and inter...,Key words: neuroglia; Extracellular Fluid; Neu...,26,319,8921,308,Switzerland
34,Frontiers in Neuroinformatics,"Poline, Jean-Baptiste",2012.0,Frontiers in Neuroinformatics,As neuroimaging databases grow in size and com...,database; Neuroimaging; neuroinformatics; pyth...,27,158,5376,149,Switzerland
23,Frontiers in Neuroanatomy,"Rockland, Kathleen S",2010.0,Frontiers in Neuroanatomy,"Abstract &ldquo;Column,&rdquo; like &ldquo;gen...",distributed; divergence; honeycomb; ocular dom...,30,424,9434,397,Switzerland
29,Frontiers in Neuroscience,"Seo, Hyojung",2010.0,Frontiers in Neuroscience,Ambivalent Dopamine,Ambivalent; Dopamine,33,1884,59190,1766,Switzerland
0,Frontiers in Computational Neuroscience,"Josic, Kresimir",2010.0,Frontiers in Computational Neuroscience,Correlations between spike trains can strongly...,Correlation; feedforward networks; pooling; sy...,64,413,7186,395,Switzerland
33,Frontiers in Neuroengineering,"Marom, Shimon",2011.0,Frontiers in Neuroengineering,Responses of individual neurons to ongoing inp...,Clamp; closed-loop; Firing-rate; Neuron; thres...,98,5,0,4,Switzerland


## Data Cleaning

### Neuro Data

In [28]:
neuro_df.head()

,title,citation_author,citation_date,citation_journal_title,description,keywords
0,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,"Offen, Daniel",2019.0,Frontiers in Neuroscience,The peripheral nervous system has an intrinsic...,Human muscle progenitor cells; Neurotrophic fa...
1,The Functional Role of Striatal Cholinergic In...,"Molkov, Yaroslav I.",2019.0,Frontiers in Neural Circuits,"In this study, we explore the functional role ...",Striatum; Basal Ganglia; Reinforcement Learn...
2,Iron Metabolism of the Skeletal Muscle and Neu...,"Herman-Antosiewicz, Anna",2019.0,Frontiers in Neuroscience,Recent studies clearly indicate that endocrine...,"myokine, ; Iron,; ALS, amyotrophic lateral scl..."
3,Brain Map of Intrinsic Functional Flexibility ...,"Wang, Zheng",2019.0,Frontiers in Neuroscience,Emerging neuroimaging studies emphasize the dy...,dynamic brain organization; intrinsic function...
4,Low CSF melanotransferrin is associated with c...,"So, Po-Wah",2019.0,Frontiers in Neuroscience,The disruption of iron metabolism and iron tra...,Alzheimer’s disease; CSF; Iron; MCI; Melanotra...


In [54]:
neuro_df["citation_date"] = neuro_df["citation_date"].fillna(0)
neuro_df.head()

,title,citation_author,citation_date,citation_journal_title,description,keywords
0,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,"Offen, Daniel",2019.0,Frontiers in Neuroscience,The peripheral nervous system has an intrinsic...,Human muscle progenitor cells; Neurotrophic fa...
1,The Functional Role of Striatal Cholinergic In...,"Molkov, Yaroslav I.",2019.0,Frontiers in Neural Circuits,"In this study, we explore the functional role ...",Striatum; Basal Ganglia; Reinforcement Learn...
2,Iron Metabolism of the Skeletal Muscle and Neu...,"Herman-Antosiewicz, Anna",2019.0,Frontiers in Neuroscience,Recent studies clearly indicate that endocrine...,"myokine, ; Iron,; ALS, amyotrophic lateral scl..."
3,Brain Map of Intrinsic Functional Flexibility ...,"Wang, Zheng",2019.0,Frontiers in Neuroscience,Emerging neuroimaging studies emphasize the dy...,dynamic brain organization; intrinsic function...
4,Low CSF melanotransferrin is associated with c...,"So, Po-Wah",2019.0,Frontiers in Neuroscience,The disruption of iron metabolism and iron tra...,Alzheimer’s disease; CSF; Iron; MCI; Melanotra...


In [55]:
neuro_df["citation_date"].unique()

array([2019., 2016., 2017., 2015., 2018., 2014., 2013., 2010., 2011.,
       2012.,    0.])

In [61]:
neuro_df["citation_date"] = neuro_df["citation_date"].astype(int, inplace = True)

In [62]:
neuro_df.head()

,title,citation_author,citation_date,citation_journal_title,description,keywords
0,HUMAN MUSCLE PROGENITOR CELLS OVEREXPRESSING N...,"Offen, Daniel",2019,Frontiers in Neuroscience,The peripheral nervous system has an intrinsic...,Human muscle progenitor cells; Neurotrophic fa...
1,The Functional Role of Striatal Cholinergic In...,"Molkov, Yaroslav I.",2019,Frontiers in Neural Circuits,"In this study, we explore the functional role ...",Striatum; Basal Ganglia; Reinforcement Learn...
2,Iron Metabolism of the Skeletal Muscle and Neu...,"Herman-Antosiewicz, Anna",2019,Frontiers in Neuroscience,Recent studies clearly indicate that endocrine...,"myokine, ; Iron,; ALS, amyotrophic lateral scl..."
3,Brain Map of Intrinsic Functional Flexibility ...,"Wang, Zheng",2019,Frontiers in Neuroscience,Emerging neuroimaging studies emphasize the dy...,dynamic brain organization; intrinsic function...
4,Low CSF melanotransferrin is associated with c...,"So, Po-Wah",2019,Frontiers in Neuroscience,The disruption of iron metabolism and iron tra...,Alzheimer’s disease; CSF; Iron; MCI; Melanotra...


### Mental Data

In [71]:
mental_df_2 = mental_df.copy()

In [72]:
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision)
0,Afghanistan; 2016,NaN,Yes,1987,Yes,2016.0
1,Afghanistan; 2014,NaN,Yes,NaN,Yes,NaN
2,Albania; 2016,NaN,Yes,2012,Yes,2013.0
3,Albania; 2014,NaN,Yes,NaN,Yes,NaN
4,Algeria; 2016,NaN,Yes,1905,No,NaN


In [73]:
for i in mental_df_2["Country; Year"]:
    mental_df_2["country"] = i.split(";")[0] 
    mental_df_2["year"] = int(i.split(";")[1])
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year
0,Afghanistan; 2016,NaN,Yes,1987,Yes,2016.0,Zimbabwe,2014
1,Afghanistan; 2014,NaN,Yes,NaN,Yes,NaN,Zimbabwe,2014
2,Albania; 2016,NaN,Yes,2012,Yes,2013.0,Zimbabwe,2014
3,Albania; 2014,NaN,Yes,NaN,Yes,NaN,Zimbabwe,2014
4,Algeria; 2016,NaN,Yes,1905,No,NaN,Zimbabwe,2014


In [74]:
mental_df_2["Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%)"].fillna(0, inplace=True)
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014
1,Afghanistan; 2014,0.0,Yes,NaN,Yes,NaN,Zimbabwe,2014
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014
3,Albania; 2014,0.0,Yes,NaN,Yes,NaN,Zimbabwe,2014
4,Algeria; 2016,0.0,Yes,1905,No,NaN,Zimbabwe,2014


In [75]:
mental_df_2["govt_exp_mntlhos_per_totgovtexp_mntl_hlth"] = mental_df_2["Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%)"]
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0
1,Afghanistan; 2014,0.0,Yes,NaN,Yes,NaN,Zimbabwe,2014,0.0
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0
3,Albania; 2014,0.0,Yes,NaN,Yes,NaN,Zimbabwe,2014,0.0
4,Algeria; 2016,0.0,Yes,1905,No,NaN,Zimbabwe,2014,0.0


In [76]:
mental_df_2["std_alone_law_mntl_hlth"] = mental_df_2["Stand-alone law for mental health"]
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0,Yes
1,Afghanistan; 2014,0.0,Yes,NaN,Yes,NaN,Zimbabwe,2014,0.0,Yes
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0,Yes
3,Albania; 2014,0.0,Yes,NaN,Yes,NaN,Zimbabwe,2014,0.0,Yes
4,Algeria; 2016,0.0,Yes,1905,No,NaN,Zimbabwe,2014,0.0,Yes


In [77]:
mental_df_2["Year the law was enacted (latest revision)"].fillna(0, inplace=True)
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0,Yes
1,Afghanistan; 2014,0.0,Yes,0,Yes,NaN,Zimbabwe,2014,0.0,Yes
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0,Yes
3,Albania; 2014,0.0,Yes,0,Yes,NaN,Zimbabwe,2014,0.0,Yes
4,Algeria; 2016,0.0,Yes,1905,No,NaN,Zimbabwe,2014,0.0,Yes


In [78]:
def convert_yr_law_enacted(row):
    if row == "No":
        return 0
    elif row == "Yes":
        return 0
    else:
       return int(row)

In [79]:
mental_df_2["yr_law_enacted_latver"] = mental_df_2["Year the law was enacted (latest revision)"].apply(convert_yr_law_enacted)
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth,yr_law_enacted_latver
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0,Yes,1987
1,Afghanistan; 2014,0.0,Yes,0,Yes,NaN,Zimbabwe,2014,0.0,Yes,0
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0,Yes,2012
3,Albania; 2014,0.0,Yes,0,Yes,NaN,Zimbabwe,2014,0.0,Yes,0
4,Algeria; 2016,0.0,Yes,1905,No,NaN,Zimbabwe,2014,0.0,Yes,1905


In [80]:
mental_df_2["std_alone_polplan_mnth_hlth"] = mental_df_2["Stand-alone policy or plan for mental health"]
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth,yr_law_enacted_latver,std_alone_polplan_mnth_hlth
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0,Yes,1987,Yes
1,Afghanistan; 2014,0.0,Yes,0,Yes,NaN,Zimbabwe,2014,0.0,Yes,0,Yes
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0,Yes,2012,Yes
3,Albania; 2014,0.0,Yes,0,Yes,NaN,Zimbabwe,2014,0.0,Yes,0,Yes
4,Algeria; 2016,0.0,Yes,1905,No,NaN,Zimbabwe,2014,0.0,Yes,1905,No


In [81]:
mental_df_2["Publication year of the policy or plan (latest revision)"].fillna(0, inplace=True)
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth,yr_law_enacted_latver,std_alone_polplan_mnth_hlth
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0,Yes,1987,Yes
1,Afghanistan; 2014,0.0,Yes,0,Yes,0.0,Zimbabwe,2014,0.0,Yes,0,Yes
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0,Yes,2012,Yes
3,Albania; 2014,0.0,Yes,0,Yes,0.0,Zimbabwe,2014,0.0,Yes,0,Yes
4,Algeria; 2016,0.0,Yes,1905,No,0.0,Zimbabwe,2014,0.0,Yes,1905,No


In [82]:
def convert_yr_to_int(row):
    return int(row)

In [83]:
mental_df_2["pub_yr_polplan_latver"] = mental_df_2["Publication year of the policy or plan (latest revision)"].apply(convert_yr_to_int)
mental_df_2.head()

,Country; Year,Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%),Stand-alone law for mental health,Year the law was enacted (latest revision),Stand-alone policy or plan for mental health,Publication year of the policy or plan (latest revision),country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth,yr_law_enacted_latver,std_alone_polplan_mnth_hlth,pub_yr_polplan_latver
0,Afghanistan; 2016,0.0,Yes,1987,Yes,2016.0,Zimbabwe,2014,0.0,Yes,1987,Yes,2016
1,Afghanistan; 2014,0.0,Yes,0,Yes,0.0,Zimbabwe,2014,0.0,Yes,0,Yes,0
2,Albania; 2016,0.0,Yes,2012,Yes,2013.0,Zimbabwe,2014,0.0,Yes,2012,Yes,2013
3,Albania; 2014,0.0,Yes,0,Yes,0.0,Zimbabwe,2014,0.0,Yes,0,Yes,0
4,Algeria; 2016,0.0,Yes,1905,No,0.0,Zimbabwe,2014,0.0,Yes,1905,No,0


In [84]:
# Drop

mental_df_2 = mental_df_2.drop(["Country; Year"], axis=1)
mental_df_2 = mental_df_2.drop(["Government expenditures on mental hospitals as a percentage of total government expenditures on mental health (%)"], axis=1)
mental_df_2 = mental_df_2.drop(["Stand-alone law for mental health"], axis=1)
mental_df_2 = mental_df_2.drop(["Year the law was enacted (latest revision)"], axis=1)
mental_df_2 = mental_df_2.drop(["Stand-alone policy or plan for mental health"], axis=1)
mental_df_2 = mental_df_2.drop(["Publication year of the policy or plan (latest revision)"], axis=1)
mental_df_2.head()

,country,year,govt_exp_mntlhos_per_totgovtexp_mntl_hlth,std_alone_law_mntl_hlth,yr_law_enacted_latver,std_alone_polplan_mnth_hlth,pub_yr_polplan_latver
0,Zimbabwe,2014,0.0,Yes,1987,Yes,2016
1,Zimbabwe,2014,0.0,Yes,0,Yes,0
2,Zimbabwe,2014,0.0,Yes,2012,Yes,2013
3,Zimbabwe,2014,0.0,Yes,0,Yes,0
4,Zimbabwe,2014,0.0,Yes,1905,No,0


### Journal Data

In [64]:
journal_df.head()

,title,Rank,Total Docs. (3years),Total Refs.,Citable Docs. (3years),Country
0,Nature Reviews Neuroscience,1,401,9159,194,United Kingdom
1,Nature Neuroscience,2,815,11676,712,United Kingdom
2,Annual Review of Neuroscience,3,67,3689,67,United States
3,Neuron,4,1560,31105,1488,United States
4,EMBO Journal,5,754,14304,660,Germany


In [65]:
journal_df.columns = [["title","rank","tot_docs","tot_refs","citable_docs","country"]]

In [66]:
journal_df.head()

,title,rank,tot_docs,tot_refs,citable_docs,country
0,Nature Reviews Neuroscience,1,401,9159,194,United Kingdom
1,Nature Neuroscience,2,815,11676,712,United Kingdom
2,Annual Review of Neuroscience,3,67,3689,67,United States
3,Neuron,4,1560,31105,1488,United States
4,EMBO Journal,5,754,14304,660,Germany


## Get String Lengths

In [68]:
def string_length(df, column):
    length = []
    for index in df[column]:
        l1 = (len(str(index)))
        length.append(l1)
    print(f"{column}: {max(length)}")

In [89]:
neuro_df.dtypes

title                     object
citation_author           object
citation_date              int64
citation_journal_title    object
description               object
keywords                  object
dtype: object

In [69]:
string_length(neuro_df,"title")
string_length(neuro_df,"citation_author")
string_length(neuro_df,"citation_journal_title")
string_length(neuro_df,"description")
string_length(neuro_df,"keywords")

title: 200
citation_author: 51
citation_journal_title: 39
description: 2853
keywords: 807


In [87]:
mental_df_2.dtypes

country                                       object
year                                           int64
govt_exp_mntlhos_per_totgovtexp_mntl_hlth    float64
std_alone_law_mntl_hlth                       object
yr_law_enacted_latver                          int64
std_alone_polplan_mnth_hlth                   object
pub_yr_polplan_latver                          int64
dtype: object

In [88]:
string_length(mental_df_2,"country")
string_length(mental_df_2,"std_alone_law_mntl_hlth")
string_length(mental_df_2,"std_alone_polplan_mnth_hlth")

country: 8
std_alone_law_mntl_hlth: 5
std_alone_polplan_mnth_hlth: 7


In [90]:
journal_df.dtypes

title           object
rank             int64
tot_docs         int64
tot_refs         int64
citable_docs     int64
country         object
dtype: object

In [92]:
string_length(journal_df,"title")
string_length(journal_df,"country")

title: 10
country: 12


## SQL Connection

In [43]:
eng = create_engine('postgresql+psycopg2://'+ username +':' + pgpass + '@' +host_loc + ':' + cred_port + '/' + db)
conn = eng.connect()

In [45]:
print(eng.table_names())

['neuroscience', 'mental_health_pol']


In [46]:
neuro_df.to_sql(name='neuroscience', con=eng, if_exists='append', index=False)

In [47]:
mental_df_2.to_sql(name='mental_health_pol', con=eng, if_exists='append', index=False)

In [ ]:
journal_df.to_sql(name='mental_health_pol', con=eng, if_exists='append', index=False)